# Devils DFTD 2-Stage Infection

In [1]:
import os
import sys
sys.path.insert(1, os.path.abspath(os.path.join(os.getcwd(), '../../GillesPy2')))

In [2]:
import time
import json
import pickle

In [3]:
from gillespy2 import TauHybridCSolver

In [4]:
from dask.distributed import Client
from dask import delayed

In [5]:
from Devils_DFTD_2_Stage_Infection import DevilsDFTD2StageInfection
DevilsDFTD2StageInfectionWithVaccination = DevilsDFTD2StageInfection
DevilsDFTD2StageInfectionWithImmunity = DevilsDFTD2StageInfection
DevilsDFTD2StageInfectionWithCulling = DevilsDFTD2StageInfection

from Simulation import Simulation, devil_pop
from ParameterSweep import ParameterSweep

In [6]:
c = Client(n_workers=8, threads_per_worker=1)
# c = Client("james.cs.unca.edu:12345")

## With Immunity Intervention
### Load existing state

In [ ]:
immun_state_path = "./DevilsDFTD2StageInfectionWithImmunityState.p"
if os.path.exists(immun_state_path):
    with open(immun_state_path, "rb") as immun_state_file:
        immun_state = pickle.load(immun_state_file)
    
    # Model
    immun_model = immun_state['model']
    # Simulation
    immun_sim = Simulation.load_state(immun_state['sim'])
    # Parameter Sweep Job
    immun_job = ParameterSweep.load_state(immun_state['job'])
else:
    immun_state = None
    print("No existing immunity state found.")
    
    # Model
    immun_model = DevilsDFTD2StageInfection(devil_pop, interventions="immunity")
    # Simulation
    immun_sim = Simulation(model=immun_model)
    # Parameter Sweep Job
    immun_job = ParameterSweep(model=immun_model)

### Solver

In [ ]:
immun_sol = delayed(TauHybridCSolver)(model=immun_model, variable=True)

### Simulation

In [ ]:
immun_sim.configure(solver=immun_sol)
%time _ = immun_sim.run(use_existing_results=True)

In [ ]:
immun_sim.output_dftd_devils_probs(verbose=False)

### Visualize Simulation Results

In [ ]:
immun_sim.plot()

### Parameter Sweep

In [ ]:
immun_params = [
    {"parameter": "immunity_growth_rate", "range": [0.005, 0.0075, 0.01]},
#     {"parameter": "immunity_growth_rate", "range": [0.0075, 0.01, 0.0125]},
#     {"parameter": "immunity_max_level", "range": [50, 75, 100]},
    {"parameter": "immunity_start", "range": [384, 444, 504]}
]

In [ ]:
new_params = immun_job.params != immun_params
%time immun_job.run(solver=immun_sol, params=immun_params)

In [ ]:
# if immun_state is None or new_params:
#     tic=time.time()
#     immun_job.load_data()
#     print(f"loading immunity parameter sweep data completed in {time.time()-tic:.2f}s")

### Visualize Parameter Sweep Results

In [ ]:
# res_sub_keys = ["immunity_max_level:100", "immunity_growth_rate:0.01"]
res_sub_keys = ["immunity_growth_rate:0.01"]
immun_job.plot_devil_dftd_extinction_over_param(
    res_sub_keys=res_sub_keys, key="immunity_start", param_label="Immunity Start", units="month"
)

In [ ]:
immun_job.explore_results()

### Store Immunity State

In [ ]:
# immun_state = {"model": immun_model, "sim":immun_sim, "job":immun_job}
# with open(immun_state_path, "wb") as immun_state_file:
#     pickle.dump(immun_state, immun_state_file)

## With Vaccination Intervention
### Load existing state

In [ ]:
vacc_state_path = "./DevilsDFTD2StageInfectionWithVaccinationState.p"
if os.path.exists(vacc_state_path):
    with open(vacc_state_path, "rb") as vacc_state_file:
        vacc_state = pickle.load(vacc_state_file)
        
#     # Model
    vacc_model = vacc_state['model']
#     # Simulation
#     vacc_sim = Simulation.load_state(vacc_state['sim'])
#     # Parameter Sweep Job
#     vacc_job = ParameterSweep.load_state(vacc_state['job'])
# else:
#     vacc_state = None
#     print("No existing vaccination state found.")

#     # Model
#     vacc_model = DevilsDFTD2StageInfection(devil_pop, interventions="vaccination")
#     # Simulation
#     vacc_sim = Simulation(model=vacc_model)
#     # Parameter Sweep Job
#     vacc_job = ParameterSweep(model=vacc_model)
    
# vacc_state = None
# vacc_model = DevilsDFTD2StageInfection(devil_pop, interventions="vaccination")
vacc_sim = Simulation(model=vacc_model)
vacc_job = ParameterSweep(model=vacc_model)

### Solver

In [ ]:
vacc_sol = delayed(TauHybridCSolver)(model=vacc_model, variable=True)

### Simulation

In [ ]:
vacc_sim.configure(solver=vacc_sol)
%time _ = vacc_sim.run(use_existing_results=True, verbose=True)

In [ ]:
vacc_sim.output_dftd_devils_probs(verbose=False)

### Visualize Simulation Results

In [ ]:
vacc_sim.plot()

### Parameter Sweep

In [ ]:
params = [
        {"parameter": "vaccinated_infection_rate", "range": [0.1, 0.2, 0.4, 0.6]},
        {"parameter": "vaccination_proportion", "range": [0.6, 0.8, 1.0]},
        {"parameter": "vacc_program_length", "range": [3, 5, 10]},
        {"parameter": "vaccine_frequency", "range": [2, 4, 6]}
    ]

In [ ]:
new_params = vacc_job.params != params
%time vacc_job.run(solver=vacc_sol, params=params, verbose=True)

In [ ]:
# if vacc_state is None or new_params:
#     tic=time.time()
#     vacc_job.load_data()
#     print(f"loading vaccination parameter sweep data completed in {time.time()-tic:.2f}s")

In [ ]:
for sim in vacc_job.results.values():
    print(sim.dftd_elimination, sim.devil_extinction)

### Visualize Parameter Sweep Results

In [ ]:
res_sub_keys=["vaccine_frequency:6", "vaccination_proportion:0.8", "vaccinated_infection_rate:0.6"]
vacc_job.plot_devil_dftd_extinction_over_param(
    res_sub_keys=res_sub_keys, key="vacc_program_length", param_label="Vaccine Program Length", units="years"
)

In [ ]:
vacc_job.explore_results()

### Store Vaccination State

In [ ]:
# vacc_state = {"model": vacc_model, "sim":vacc_sim, "job":vacc_job}
# with open(vacc_state_path, "wb") as vacc_state_file:
#     pickle.dump(vacc_state, vacc_state_file)

## With Culling Intervention
### Load existing state

In [ ]:
cull_state_path = "./DevilsDFTD2StageInfectionWithCullingState.p"
if os.path.exists(cull_state_path):
    with open(cull_state_path, "rb") as cull_state_file:
        cull_state = pickle.load(cull_state_file)
        
    # Model
    cull_model = cull_state['model']
    # Simulation
    cull_sim = Simulation.load_state(cull_state['sim'])
    # Parameter Sweep Job
    cull_job = ParameterSweep.load_state(cull_state['job'])
else:
    cull_state = None
    print("No existing vaccination state found.")

    # Model
    cull_model = DevilsDFTD2StageInfection(devil_pop, interventions="culling")
    # Simulation
    cull_sim = Simulation(model=cull_model)
    # Parameter Sweep Job
    cull_job = ParameterSweep(model=cull_model)

### Solver

In [ ]:
cull_sol = delayed(TauHybridCSolver)(model=cull_model, variable=True)

### Simulation

In [ ]:
cull_sim.configure(solver=cull_sol)
%time _ = cull_sim.run(use_existing_results=True)

In [ ]:
cull_sim.output_dftd_devils_probs(verbose=False)

### Visualize Simulation Results

In [ ]:
cull_sim.plot()

### Parameter Sweep

In [ ]:
params = [
    {"parameter": "cull_rate_diseased", "range": [0.25, 0.5, 0.75]},
    {"parameter": "cull_program_length", "range": [3, 5, 10]}
]

In [ ]:
new_params = cull_job.params != params
%time cull_job.run(solver=cull_sol, params=params, verbose=True)

In [ ]:
# if cull_state is None or new_params:
#     tic=time.time()
#     cull_job.load_data()
#     print(f"loading culling parameter sweep data completed in {time.time()-tic:.2f}s")

### Visualize Parameter Sweep Results

In [ ]:
res_sub_keys = ["cull_rate_diseased:0.5"]
cull_job.plot_devil_dftd_extinction_over_param(
    res_sub_keys=res_sub_keys, key="cull_program_length", param_label="Culling Program Length", units="years"
)

In [ ]:
cull_job.explore_results()

### Store Culling State

In [ ]:
# cull_state = {"model": cull_model, "sim":cull_sim, "job":cull_job}
# with open(cull_state_path, "wb") as cull_state_file:
#     pickle.dump(cull_state, cull_state_file)